# Exploring and clustering Toronto Neighborhood data

### Introduction:
    In this notebook, I will be using Toronto Neighborhood data and create data exploration and clusters of the neighborhood. I will be using Folium package to visualize the data and K-means for cluster analysis.

Downloading all the required packages

In [7]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                       

### 1. Download and explore Toronto data
       We will download the neighborhood data for Toronto from the following link 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
       Once downloaded we will do sanity checks and perform clean up activities if needed. After cleaning up the data we will explore and visualize  

Read data from Wiki page

In [8]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', na_values = 'Not assigned', flavor = 'html5lib')
df =df[0]

In [9]:
type(df)

pandas.core.frame.DataFrame

In [10]:
df.rename({"Postal code": 'Postal_cd', 1: 'Borough', 2: 'Neighbourhood'}, axis = 'columns', inplace = True)

In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal_cd       180 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


There are null values in Borough and Neighborhood columns. We will take a look at them.

In [12]:
df.isnull().sum()

Postal_cd        0
Borough         77
Neighborhood    77
dtype: int64

In [13]:
df[df.Postal_cd == 'M5A']

,Postal_cd,Borough,Neighborhood
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Drop the rows that has NAN values in Borough column

In [14]:
df.dropna(subset=['Borough'],inplace=True)
df.isnull().sum()

Postal_cd       0
Borough         0
Neighborhood    0
dtype: int64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 2 to 178
Data columns (total 3 columns):
Postal_cd       103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB


In [16]:
df[df.Postal_cd == 'M5V']

,Postal_cd,Borough,Neighborhood
139,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


### Cleanup the data frame to split multiple neighborhood values into separate rows

In [17]:
new_df = pd.DataFrame(df.Neighborhood.str.split('/').tolist(), index=df.Postal_cd).stack()
new_df = new_df.reset_index([0, 'Postal_cd'])
new_df.columns = ['Postal_cd', 'Neighborhood']
df_merge_col = pd.merge(df, new_df, left_on='Postal_cd', right_on='Postal_cd')
df_merge_col.drop('Neighborhood_x',axis=1,inplace=True)
newcols = {
    'Postal_cd': 'Postal_cd', 
    'Borough': 'Borough', 
    'Neighborhood_y': 'Neighborhood'
}
df_merge_col.rename(columns=newcols,inplace=True)
df_merge_col= df_merge_col[['Postal_cd','Borough','Neighborhood']]

In [18]:
df_merge_col.head()

,Postal_cd,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor


In [19]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(

len(df_merge_col['Borough'].unique()),

len(df_merge_col['Neighborhood'].unique())))
print(df_merge_col.shape)

The dataframe has 10 boroughs and 202 neighbourhoods.
(208, 3)


In [20]:
# The code was removed by Watson Studio for sharing.

,Postal_cd,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge Dataframes to create one with Latitude and Longitude information

In [21]:
df_Loc = pd.merge(df_merge_col, df_data_1, left_on='Postal_cd', right_on='Postal_cd')

In [22]:
df_Loc.head()

,Postal_cd,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


Visulize Boroughs in Toronto using Folium

In [23]:
map_toronto = folium.Map(location=[43.654260, -79.360636], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Loc['Latitude'], df_Loc['Longitude'], df_Loc['Borough'], df_Loc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [24]:
Toronto_data = df_Loc[df_Loc['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data.head()

,Postal_cd,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
1,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
2,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
3,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [25]:
map_toronto_dt = folium.Map(location=[43.654260, -79.360636], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_dt)  
    
map_toronto_dt

Foursquare Creds Defintion

In [26]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: IOCGLJ4A4STLADHSAYEXJRWN4GB1DNGFFIPE14BRX0A34WZM
CLIENT_SECRET:HUGG2ZDFVGCQVXVL5A5E1S32GVECXBAD2R3OE2H4OVO3DFNU


In [27]:
Toronto_data.loc[1,'Neighborhood']

' Harbourfront'

Get the neighborhood's latitude and longitude values.

In [28]:
neighborhood_latitude = Toronto_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of  Harbourfront are 43.6542599, -79.3606359.


In [29]:
# create URL
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=IOCGLJ4A4STLADHSAYEXJRWN4GB1DNGFFIPE14BRX0A34WZM&client_secret=HUGG2ZDFVGCQVXVL5A5E1S32GVECXBAD2R3OE2H4OVO3DFNU&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

Send the GET request and examine the resutls

In [30]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8e5ec30de0d90023a8f5ff'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 45,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149


In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


Define Function to process all neighborhoods

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude'])


Regent Park 
 Harbourfront
Queen's Park 
 Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond 
 Adelaide 
 King
Dufferin 
 Dovercourt Village
Harbourfront East 
 Union Station 
 Toronto Islands
Little Portugal 
 Trinity
The Danforth West 
 Riverdale
Toronto Dominion Centre 
 Design Exchange
Brockton 
 Parkdale Village 
 Exhibition Place
India Bazaar 
 The Beaches West
Commerce Court 
 Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park 
 The Junction South
North Toronto West
The Annex 
 North Midtown 
 Yorkville
Parkdale 
 Roncesvalles
Davisville
University of Toronto 
 Harbord
Runnymede 
 Swansea
Moore Park 
 Summerhill East
Kensington Market 
 Chinatown 
 Grange Park
Summerhill West 
 Rathnelly 
 South Hill 
 Forest Hill SE 
 Deer Park
CN Tower 
 King and Spadina 
 Railway Lands 
 Harbourfront West 
 Bathurst  Quay 
 South Niagara 
 Island airport
Rosed

In [37]:
                                  
print(Toronto_venues.shape)
Toronto_venues.head()

(3112, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [38]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,100,100,100,100,100,100
Bathurst Quay,16,16,16,16,16,16
Cabbagetown,46,46,46,46,46,46
Chinatown,71,71,71,71,71,71
Deer Park,16,16,16,16,16,16
Design Exchange,100,100,100,100,100,100
Dovercourt Village,16,16,16,16,16,16
Exhibition Place,22,22,22,22,22,22
Forest Hill SE,16,16,16,16,16,16


In [39]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


### Analyze Neighborhood

In [40]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
manhattan_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Regent Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [41]:
Toronto_onehot.shape

(3112, 235)

In [42]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.020000,0.000000,0.00,0.010000,0.000000,0.020000,0.0000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.020000,0.000000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.010000,0.010000,0.000000,0.0000,0.000000,0.0000,0.050000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.090000,0.000000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.000000,0.020000,0.000,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,

In [43]:
Toronto_grouped.shape

(72, 235)

In [44]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide ----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.05
2         Café  0.05
3          Gym  0.04
4        Hotel  0.03


---- Bathurst  Quay ----
              venue  freq
0   Airport Service  0.19
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4       Coffee Shop  0.06


---- Cabbagetown----
         venue  freq
0  Coffee Shop  0.09
1         Park  0.04
2          Pub  0.04
3       Bakery  0.04
4         Café  0.04


---- Chinatown ----
                   venue  freq
0                   Café  0.07
1            Coffee Shop  0.06
2  Vietnamese Restaurant  0.06
3    Dumpling Restaurant  0.04
4     Mexican Restaurant  0.04


---- Deer Park----
                 venue  freq
0                  Pub  0.12
1          Coffee Shop  0.12
2                 Bank  0.06
3  Fried Chicken Joint  0.06
4          Supermarket  0.06


---- Design Exchange----
                venue  freq
0         Coffee Shop  0.12
1               Hotel  0.08
2        

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Café,Gym,Thai Restaurant,Hotel,Bakery,Vegetarian / Vegan Restaurant,American Restaurant,Steakhouse
1,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Coffee Shop,Rental Car Location,Harbor / Marina
2,Cabbagetown,Coffee Shop,Park,Bakery,Chinese Restaurant,Pizza Place,Café,Pub,Italian Restaurant,Restaurant,Butcher
3,Chinatown,Café,Vietnamese Restaurant,Coffee Shop,Dumpling Restaurant,Bar,Mexican Restaurant,Park,Farmers Market,Dessert Shop,Chinese Restaurant
4,Deer Park,Coffee Shop,Pub,Bank,Vietnamese Restaurant,Light Rail Station,Supermarket,Liquor Store,American Restaurant,Restaurant,Sushi Restaurant


### Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [47]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 3, 3, 3, 3, 0, 0, 3, 3], dtype=int32)

In [48]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal_cd,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,3.0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Café,Breakfast Spot,Restaurant,Electronics Store,Spa
1,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,3.0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Café,Breakfast Spot,Restaurant,Electronics Store,Spa
2,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,3.0,Coffee Shop,Diner,Yoga Studio,Distribution Center,Beer Bar,Italian Restaurant,Juice Bar,Sandwich Place,Burger Joint,Restaurant
3,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494,3.0,Coffee Shop,Diner,Yoga Studio,Distribution Center,Beer Bar,Italian Restaurant,Juice Bar,Sandwich Place,Burger Joint,Restaurant
4,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3.0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Tea Room,Bookstore,Ramen Restaurant


### Examine Clusters¶


In [49]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,0.0,Grocery Store,Café,Park,Nightclub,Restaurant,Candy Store,Italian Restaurant,Diner,Baby Store,Gas Station
13,West Toronto,0.0,Pharmacy,Bakery,Park,Bus Stop,Café,Bar,Supermarket,Bank,Furniture / Home Store,Middle Eastern Restaurant
14,West Toronto,0.0,Pharmacy,Bakery,Park,Bus Stop,Café,Bar,Supermarket,Bank,Furniture / Home Store,Middle Eastern Restaurant
24,West Toronto,0.0,Café,Breakfast Spot,Coffee Shop,Convenience Store,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
25,West Toronto,0.0,Café,Breakfast Spot,Coffee Shop,Convenience Store,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
26,West Toronto,0.0,Café,Breakfast Spot,Coffee Shop,Convenience Store,Burrito Place,Restaurant,Italian Restaurant,Intersection,Stadium,Bar
31,East Toronto,0.0,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Sandwich Place,Cheese Shop,Pet Store,Park
36,West Toronto,0.0,Café,Mexican Restaurant,Bar,Thai Restaurant,Park,Fried Chicken Joint,Bookstore,Diner,Fast Food Restaurant,Italian Restaurant
37,West Toronto,0.0,Café,Mexican Restaurant,Bar,Thai Restaurant,Park,Fried Chicken Joint,Bookstore,Diner,Fast Food Restaurant,Italian Restaurant
39,Central Toronto,0.0,Sandwich Place,Café,Coffee Shop,Middle Eastern Restaurant,Pizza Place,Liquor Store,Burger Joint,Indian Restaurant,Pub,History Museum


In [50]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Downtown Toronto,1.0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Coffee Shop,Rental Car Location,Harbor / Marina
60,Downtown Toronto,1.0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Coffee Shop,Rental Car Location,Harbor / Marina
61,Downtown Toronto,1.0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Coffee Shop,Rental Car Location,Harbor / Marina
62,Downtown Toronto,1.0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Coffee Shop,Rental Car Location,Harbor / Marina
63,Downtown Toronto,1.0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Coffee Shop,Rental Car Location,Harbor / Marina
64,Downtown Toronto,1.0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Coffee Shop,Rental Car Location,Harbor / Marina
65,Downtown Toronto,1.0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Sculpture Garden,Coffee Shop,Rental Car Location,Harbor / Marina


In [51]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Central Toronto,2.0,Garden,Yoga Studio,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center


In [52]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3.0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Café,Breakfast Spot,Restaurant,Electronics Store,Spa
1,Downtown Toronto,3.0,Coffee Shop,Pub,Bakery,Park,Mexican Restaurant,Café,Breakfast Spot,Restaurant,Electronics Store,Spa
2,Downtown Toronto,3.0,Coffee Shop,Diner,Yoga Studio,Distribution Center,Beer Bar,Italian Restaurant,Juice Bar,Sandwich Place,Burger Joint,Restaurant
3,Downtown Toronto,3.0,Coffee Shop,Diner,Yoga Studio,Distribution Center,Beer Bar,Italian Restaurant,Juice Bar,Sandwich Place,Burger Joint,Restaurant
4,Downtown Toronto,3.0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Tea Room,Bookstore,Ramen Restaurant
5,Downtown Toronto,3.0,Café,Coffee Shop,Cocktail Bar,Hotel,Beer Bar,Gastropub,Restaurant,American Restaurant,Italian Restaurant,Diner
6,East Toronto,3.0,Trail,Health Food Store,Pub,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Yoga Studio,Cupcake Shop
7,Downtown Toronto,3.0,Cocktail Bar,Coffee Shop,Café,Restaurant,Farmers Market,Bakery,Seafood Restaurant,Cheese Shop,Beer Bar,Italian Restaurant
8,Downtown Toronto,3.0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Spa,Salad Place,Bubble Tea Shop,Gym / Fitness Center,Ice Cream Shop
10,Downtown Toronto,3.0,Coffee Shop,Restaurant,Café,Gym,Thai Restaurant,Hotel,Bakery,Vegetarian / Vegan Restaurant,American Restaurant,Steakhouse


In [53]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Central Toronto,4.0,Park,Bus Line,Swim School,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
66,Downtown Toronto,4.0,Park,Playground,Trail,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
